In [1]:
#Instalamos librerias necesarias (En caso de requerirlo)
#pip install requests
#pip install beautifulsoup4
#pip install pandas
#pip install selenium --upgrade

In [8]:
# Importamos librerias necesarias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3
from csv import writer
from datetime import datetime

In [9]:
# Indicar Path hacia el archivo ejecutable de geckodriver
gecko_driver_path = 'C:/Users/HP/Documents/9-Master Big Data - Daniela/(MBD-2324) Trabajo Fin de Máster O1/chromedriver-win64/chromedriver.exe'

# Preparar y abrir el navegador (Firefox)
service = Service(gecko_driver_path)
driver = webdriver.Chrome(service=service)

In [10]:
# Abrir URL de la pagina de vehiculos usados
pageUrl = "https://www.kavak.com/mx/seminuevos"
driver.get(pageUrl)

# Funcion para estructurar el URL para cada pagina
def get_page_url(pageNumber):
    base_page_url = "https://www.kavak.com/mx/seminuevos"
    return f"{base_page_url}?page={pageNumber}"

In [11]:
# Funcion para extraer los detalles de cada publicacion
def publication_object(car_publication):
    try:
        # Extraer marca y modelo
        marca_modelo = car_publication.find(class_="card-product_cardProduct__title__RR0CK").text.strip()
        marca, modelo = marca_modelo.split(" • ") if " • " in marca_modelo else (marca_modelo, None)
        # Extraer año, kilometraje, version y transmision
        subtitulo = car_publication.find(class_="card-product_cardProduct__subtitle__hbN2a").text.strip()
        año, kilometraje, version, transmision = subtitulo.split(" • ") if " • " in subtitulo else (subtitulo, None)
        
        kilometraje = kilometraje.split(" ")
        kilometraje = kilometraje[0]
        kilometraje = kilometraje.replace(",","")
        
        # Extraer precio
        precio = car_publication.find(class_="uki-amount amount_uki-amount__BDtqH amount_uki-amount__BDtqH amount_uki-amount__large__SePZX").text
        precio = precio.split("$")
        precio = precio[1]
        precio = precio.replace(",","")
        
        # Extraer ubicacion del vehiculo
        ubicacion = car_publication.find(class_="card-product_cardProduct__footer__MQKa_").text.strip()
                
        # Asignar nombre a las columnas
        return {
            "marca": marca,
            "modelo": modelo,
            "año": año,
            "kilometraje": kilometraje,
            "version": version,
            "transmision": transmision,
            "precio (MXN)": precio,
            "ubicacion": ubicacion
         }

    except Exception as e:
        print(f"Error extracting data: {e}")
        return {}

In [12]:
# Funcion para hacer scrape y guardar las publicaciones de cada pagina

def parser_and_save_items(pageUrl, is_first_page):
    driver.get(pageUrl)
    time.sleep(3)
 
    # Obtener el contenido HTML y crear un dataframe
    html_code = driver.page_source
    soup = BeautifulSoup(html_code, 'lxml')
    all_publications = soup.find_all("a", class_="card-product_cardProduct__3qZCJ")
    df = pd.DataFrame()  
       
    for car_publication in all_publications:
        data = publication_object(car_publication)
        if data: 
            df_new = pd.DataFrame([data])
            df = pd.concat([df, df_new], ignore_index=True)
               
    # Exportar la data
    export_data(df, is_first_page)
    print("Data exported for page:", pageUrl)
    
# Funcion para exportar la data a un archivo CSV
def export_data(df, is_first_page):
    df.to_csv(
        'cars_publications.csv',
        mode='a',
        encoding='utf-8-sig',
        index=False,
        header=is_first_page
    )

In [13]:
# Iterar a traves de las diferentes paginas para hacer el scraping
for actual_number_page in range(0, 150):  
    url_page = get_page_url(actual_number_page)
    print("Scraping page:", url_page)
    is_first_page = (actual_number_page == 0)  # Indica cual es la primera pagina
    parser_and_save_items(url_page, is_first_page)

# Cerrar el navegador
driver.quit()

Scraping page: https://www.kavak.com/mx/seminuevos?page=0
Data exported for page: https://www.kavak.com/mx/seminuevos?page=0
Scraping page: https://www.kavak.com/mx/seminuevos?page=1
Data exported for page: https://www.kavak.com/mx/seminuevos?page=1
Scraping page: https://www.kavak.com/mx/seminuevos?page=2
Data exported for page: https://www.kavak.com/mx/seminuevos?page=2
Scraping page: https://www.kavak.com/mx/seminuevos?page=3
Data exported for page: https://www.kavak.com/mx/seminuevos?page=3
Scraping page: https://www.kavak.com/mx/seminuevos?page=4
Data exported for page: https://www.kavak.com/mx/seminuevos?page=4
Scraping page: https://www.kavak.com/mx/seminuevos?page=5
Data exported for page: https://www.kavak.com/mx/seminuevos?page=5
Scraping page: https://www.kavak.com/mx/seminuevos?page=6
Data exported for page: https://www.kavak.com/mx/seminuevos?page=6
Scraping page: https://www.kavak.com/mx/seminuevos?page=7
Data exported for page: https://www.kavak.com/mx/seminuevos?page=7


Data exported for page: https://www.kavak.com/mx/seminuevos?page=65
Scraping page: https://www.kavak.com/mx/seminuevos?page=66
Data exported for page: https://www.kavak.com/mx/seminuevos?page=66
Scraping page: https://www.kavak.com/mx/seminuevos?page=67
Data exported for page: https://www.kavak.com/mx/seminuevos?page=67
Scraping page: https://www.kavak.com/mx/seminuevos?page=68
Data exported for page: https://www.kavak.com/mx/seminuevos?page=68
Scraping page: https://www.kavak.com/mx/seminuevos?page=69
Data exported for page: https://www.kavak.com/mx/seminuevos?page=69
Scraping page: https://www.kavak.com/mx/seminuevos?page=70
Data exported for page: https://www.kavak.com/mx/seminuevos?page=70
Scraping page: https://www.kavak.com/mx/seminuevos?page=71
Data exported for page: https://www.kavak.com/mx/seminuevos?page=71
Scraping page: https://www.kavak.com/mx/seminuevos?page=72
Data exported for page: https://www.kavak.com/mx/seminuevos?page=72
Scraping page: https://www.kavak.com/mx/sem

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N

Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'NoneType' object has no attribute 'text'
Error extracting data: 'N